https://towardsdatascience.com/multi-label-text-classification-with-scikit-learn-30714b7819c5

In [53]:
from datasets import load_dataset
import pandas as pd

from sklearn.preprocessing import MultiLabelBinarizer

In [54]:
%matplotlib inline
import re
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
import seaborn as sns

In [55]:
seed = 94487

In [56]:
dataset = load_dataset("ohsumed")

Reusing dataset ohsumed (/Users/carlosmorote/.cache/huggingface/datasets/ohsumed/ohsumed/1.1.0/dbd732a10148e768f8cd18872e9403d6fb3c67b696d32fea0a88c37041e91a03)
100%|██████████| 2/2 [00:00<00:00, 43.89it/s]


In [57]:
df = pd.DataFrame(dataset['train'])
df.sample(5, random_state=seed)

,seq_id,medline_ui,mesh_terms,title,publication_type,abstract,author,source
31186,1,87323006,"Atrioventricular Node/*PH; Cardiac Pacing, Art...",Effects of upright posture on anterograde and ...,JOURNAL ARTICLE.,To assess the effects of posture on anterograd...,Mann DE; Sensecqua JE; Easley AR; Reiter MJ.,Am J Cardiol 8712; 60(7):625-9
14591,1,87177986,Animal; Comparative Study; Evolution; Locomoti...,Bipedal vertebrates [letter],LETTER.,The somatomedins or insulin-like growth factor...,Farlow JO.,Science 8707; 236(4799):243
39032,1,87320047,Acute Disease; Age Factors; Cardiopulmonary By...,Intra-abdominal complications of cardiopulmona...,JOURNAL ARTICLE.,"The records of 6,452 consecutive patients who ...",Leitman IM; Paull DE; Barie PS; Isom OW; Shire...,Surg Gynecol Obstet 8712; 165(3):251-4
1690,1,87065021,"Africa; Human; Models, Genetic/*; Phylogeny/*.",Occam and mankind's genetic bottleneck [letter],LETTER.,The value of skull radiography in identifying ...,Greenwood JJ.,Nature 8703; 324(6092):21-2
20357,1,87225196,"Bedding and Linens/*; Laundry Service, Hospita...","Monitoring of radioactive ""dirty linen"" after ...",LETTER.,The presence of normal biliary flow was establ...,Marcus CS; Karbus JE.,J Nucl Med 8709; 28(6):1064


In [60]:
df = df.drop(columns=['seq_id','medline_ui','title','publication_type','author','source'])
df.sample(5, random_state=seed)

,mesh_terms,abstract
31186,"Atrioventricular Node/*PH; Cardiac Pacing, Art...",To assess the effects of posture on anterograd...
14591,Animal; Comparative Study; Evolution; Locomoti...,The somatomedins or insulin-like growth factor...
39032,Acute Disease; Age Factors; Cardiopulmonary By...,"The records of 6,452 consecutive patients who ..."
1690,"Africa; Human; Models, Genetic/*; Phylogeny/*.",The value of skull radiography in identifying ...
20357,"Bedding and Linens/*; Laundry Service, Hospita...",The presence of normal biliary flow was establ...


In [39]:
df['mesh_terms'] = df['mesh_terms'].str.split(';')
df.sample(5, random_state=seed)

,mesh_terms,abstract
31186,"[Atrioventricular Node/*PH, Cardiac Pacing, A...",To assess the effects of posture on anterograd...
14591,"[Animal, Comparative Study, Evolution, Loco...",The somatomedins or insulin-like growth factor...
39032,"[Acute Disease, Age Factors, Cardiopulmonary...","The records of 6,452 consecutive patients who ..."
1690,"[Africa, Human, Models, Genetic/*, Phylogen...",The value of skull radiography in identifying ...
20357,"[Bedding and Linens/*, Laundry Service, Hospi...",The presence of normal biliary flow was establ...


In [40]:
df['mesh_terms'] = df['mesh_terms'].apply(lambda terms: [re.sub(r'(/.*|\*|\.)*','',term) for term in terms])
df.sample(5, random_state = seed)

,mesh_terms,abstract
31186,"[Atrioventricular Node, Cardiac Pacing, Artif...",To assess the effects of posture on anterograd...
14591,"[Animal, Comparative Study, Evolution, Loco...",The somatomedins or insulin-like growth factor...
39032,"[Acute Disease, Age Factors, Cardiopulmonary...","The records of 6,452 consecutive patients who ..."
1690,"[Africa, Human, Models, Genetic, Phylogeny]",The value of skull radiography in identifying ...
20357,"[Bedding and Linens, Laundry Service, Hospita...",The presence of normal biliary flow was establ...


In [41]:
s = df['mesh_terms']

mlb = MultiLabelBinarizer()

df_aux = pd.DataFrame(mlb.fit_transform(s),columns=mlb.classes_, index=df.index)

In [42]:
df_aux.columns

Index([' 1,2-Dipalmitoylphosphatidylcholine', ' 1-Carboxyglutamic Acid',
       ' 1-Methyl-3-Isobutylxanthine',
       ' 1-Sarcosine-8-Isoleucine Angiotensin II',
       ' 16,16-Dimethylprostaglandin E2', ' 17-Hydroxycorticosteroids',
       ' 17-Hydroxysteroid Dehydrogenases', ' 17-Ketosteroids',
       ' 18-Hydroxydesoxycorticosterone',
       ' 2',3'-Cyclic Nucleotide Phosphodiesterases',
       ...
       'beta-Lactamases', 'p-Aminoazobenzene', 'p-Aminohippuric Acid',
       'p-Aminosalicylic Acid', 'p-Azobenzenearsonate', 'p-Chloroamphetamine',
       'p-Chlorophenylalanine', 'p-Hydroxyamphetamine',
       'von Willebrand Factor', 'von Willebrand's Disease'],
      dtype='object', length=14201)

In [43]:
df_aux['abstract'] = df['abstract']
df = df_aux
del df_aux
df.sample(5, random_state=seed)

,"1,2-Dipalmitoylphosphatidylcholine",1-Carboxyglutamic Acid,1-Methyl-3-Isobutylxanthine,1-Sarcosine-8-Isoleucine Angiotensin II,"16,16-Dimethylprostaglandin E2",17-Hydroxycorticosteroids,17-Hydroxysteroid Dehydrogenases,17-Ketosteroids,18-Hydroxydesoxycorticosterone,"2',3'-Cyclic Nucleotide Phosphodiesterases",...,p-Aminoazobenzene,p-Aminohippuric Acid,p-Aminosalicylic Acid,p-Azobenzenearsonate,p-Chloroamphetamine,p-Chlorophenylalanine,p-Hydroxyamphetamine,von Willebrand Factor,von Willebrand's Disease,abstract
31186,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,To assess the effects of posture on anterograd...
14591,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,The somatomedins or insulin-like growth factor...
39032,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"The records of 6,452 consecutive patients who ..."
1690,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,The value of skull radiography in identifying ...
20357,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,The presence of normal biliary flow was establ...


In [44]:
s_sum = df.drop(columns=['abstract']).sum()
df = df.drop(columns=s_sum[s_sum < 30].keys())
df

,6-Ketoprostaglandin F1 alpha,AIDS-Related Complex,Acute Disease,Adenoma,Adenosine Triphosphate,"Administration, Oral",Adolescence,Adolescent Behavior,Adrenal Cortex Hormones,Adrenal Gland Neoplasms,...,Female,Great Britain,"History of Medicine, 20th Cent",Human,alpha 1-Antitrypsin,alpha Fetoproteins,beta 2-Microglobulin,beta-Lactamases,von Willebrand Factor,abstract
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Some patients converted from ventricular fibri...
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,There is controversy regarding the appropriate...
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,As the use of helicopters for air transport of...
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,A prospective clinical trial was conducted to ...
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,An unusual case of a misdirected nasogastric t...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54704,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Lupus anticoagulants and anticardiolipin antib...
54705,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Lupus anticoagulants and anticardiolipin antib...
54706,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Vinculin is a specialized protein with a molec...
54707,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Intraocular fibrin formation following ocular ...


In [45]:
c_sum = df.drop(columns=['abstract']).sum(axis=1)
c_sum_index = c_sum[c_sum == 0].index

In [46]:
df = df.drop(index=c_sum_index)
df#.sample(5, random_state=seed)

,6-Ketoprostaglandin F1 alpha,AIDS-Related Complex,Acute Disease,Adenoma,Adenosine Triphosphate,"Administration, Oral",Adolescence,Adolescent Behavior,Adrenal Cortex Hormones,Adrenal Gland Neoplasms,...,Female,Great Britain,"History of Medicine, 20th Cent",Human,alpha 1-Antitrypsin,alpha Fetoproteins,beta 2-Microglobulin,beta-Lactamases,von Willebrand Factor,abstract
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Some patients converted from ventricular fibri...
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,There is controversy regarding the appropriate...
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,As the use of helicopters for air transport of...
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,A prospective clinical trial was conducted to ...
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,An unusual case of a misdirected nasogastric t...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54704,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Lupus anticoagulants and anticardiolipin antib...
54705,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Lupus anticoagulants and anticardiolipin antib...
54706,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Vinculin is a specialized protein with a molec...
54707,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Intraocular fibrin formation following ocular ...


In [52]:
sorted(df.drop(columns=['abstract']).sum())[-10:]

[5522, 6486, 7668, 9418, 9526, 12058, 13406, 20198, 20849, 43122]

In [25]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "can not ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    text = re.sub('\W', ' ', text)
    text = re.sub('\s+', ' ', text)
    text = text.strip(' ')
    return text

In [26]:
df['abstract'] = df['abstract'].map(lambda com : clean_text(com))

In [28]:
df['abstract'].sample(5, random_state=seed)

46227    amplification of the human protooncogene c erb...
10190    the efficacy safety and mechanisms of penicill...
44930    in this review i have briefly outlined some of...
6887     the authors studied the single pass pulmonary ...
38302    conversion of t4 to t3 cannot be demonstrated ...
Name: abstract, dtype: object

In [29]:
vec = TfidfVectorizer(stop_words=stop_words)

In [30]:
clf = OneVsRestClassifier(MultinomialNB(
                    fit_prior=True, class_prior=None))

In [31]:
NB_pipeline = Pipeline([
                ('tfidf', TfidfVectorizer(stop_words=stop_words)),
                ('clf', OneVsRestClassifier(MultinomialNB(
                    fit_prior=True, class_prior=None))),
            ])

In [32]:
for category in df.drop(columns=['abstract']).columns:
    NB_pipeline.fit(df['abstract'],df[category])

KeyboardInterrupt: 